In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
import metal
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
from dataset import STSBDataset

In [6]:
from BERT_tasks import create_task
tasks = [create_task('STSB')]

Loading STSB Dataset


100%|██████████| 1500/1500 [00:00<00:00, 1786.60it/s]


In [8]:
%%time

from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model,
    tasks,
    n_epochs=3,
    lr=0.0001, l2=0,
    progress_bar=True,
    log_every=100,
    score_every=200,
    log_unit="batches",
    checkpoint_best=True,
    checkpoint_metric="STSB/valid/spearman_corr",
    checkpoint_metric_mode="max",
)

Beginning train loop.
Expecting a total of 5749 examples and 1150 batches per epoch from 1 tasks.


[ (0.04 epo)]: TRAIN:[loss=0.115]
[ (0.09 epo)]: TRAIN:[loss=0.086] VALID:[STSB/pearson_corr=0.3872022032737732, STSB/spearman_corr=0.384]
Saving model at iteration 100 with best (max) score 0.384
[ (0.13 epo)]: TRAIN:[loss=0.100]
[ (0.17 epo)]: TRAIN:[loss=0.077] VALID:[STSB/pearson_corr=0.3145037889480591, STSB/spearman_corr=0.346]
[ (0.22 epo)]: TRAIN:[loss=0.140]
[ (0.26 epo)]: TRAIN:[loss=0.097] VALID:[STSB/pearson_corr=0.1650836318731308, STSB/spearman_corr=0.187]
[ (0.30 epo)]: TRAIN:[loss=0.093]
[ (0.35 epo)]: TRAIN:[loss=0.086] VALID:[STSB/pearson_corr=0.22912976145744324, STSB/spearman_corr=0.267]
[ (0.39 epo)]: TRAIN:[loss=0.101]
[ (0.43 epo)]: TRAIN:[loss=0.082] VALID:[STSB/pearson_corr=-0.031053977087140083, STSB/spearman_corr=-0.023]
[ (0.48 epo)]: TRAIN:[loss=0.105]
[ (0.52 epo)]: TRAIN:[loss=0.095] VALID:[STSB/pearson_corr=0.2657110393047333, STSB/spearman_corr=0.276]
[ (0.57 epo)]: TRAIN:[loss=0.089]
[ (0.61 epo)]: TRAIN:[loss=0.104] VALID:[STSB/pearson_corr=0.00140266

KeyboardInterrupt: 